https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=code-example

In [1]:
import os
from typing import Annotated

from semantic_kernel.agents.open_ai import AzureAssistantAgent
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents.streaming_file_reference_content import StreamingFileReferenceContent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

from semantic_kernel.core_plugins.web_search_engine_plugin import WebSearchEnginePlugin

from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

from semantic_kernel.kernel import Kernel

from dotenv import load_dotenv

load_dotenv(override=True)

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_chat_model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")  
bing_search_key = os.getenv("BING_SEARCH_API_KEY")

In [2]:
kernel = Kernel()

service_id = "bing_agent"

# adding openai service to kernel
chat_service = AzureChatCompletion(
    service_id=service_id,
    api_key=azure_openai_key,
    endpoint=azure_openai_endpoint,
    deployment_name=azure_openai_chat_model,
)
kernel.add_service(chat_service)

connector = BingConnector(api_key=bing_search_key)
kernel.add_plugin(WebSearchEnginePlugin(connector), plugin_name="WebSearch")

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# creating code interpreter agent
code_agent = await AzureAssistantAgent.create(
    kernel=kernel,
    service_id=service_id,
    name="BingAgent",
    instructions="you are helpful agent that can get answers from the web, use webSearch plugin to get the answer",
    execution_settings=settings,

    # enable_code_interpreter=True
)

print("Creating thread...")
thread_id = await code_agent.create_thread()

user_input = "what was top 5 news on 2025-01-21?"

await code_agent.add_chat_message(
    thread_id=thread_id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
)

async for content in code_agent.invoke(thread_id=thread_id):
    print(f"# {content.role} - {content.name or '*'}: '{content.content}'")


Creating thread...
# AuthorRole.ASSISTANT - BingAgent: 'Here are the top news highlights from January 21, 2025:

1. **Israeli General Resignation**: Israel's top general resigned, citing security and intelligence failures related to Hamas, which has raised concerns about the country's military leadership and strategy.

2. **Gaza Casualties**: Civil defense workers in Gaza reported the discovery of 137 bodies buried under the rubble of homes, schools, and businesses following recent conflicts.

3. **U.S. Executive Orders**: On January 20, 2025, U.S. President Donald Trump issued a series of executive orders and directives focused on various issues, including energy policies, as he sought to establish his administration's priorities.

4. **General News Coverage**: Various news outlets provided updates and analysis of the significant events occurring on this date, including reactions to the Israeli general's resignation and the situation in Gaza.

5. **Military Equipment Updates**: There 